In [60]:
!pip install langchain-huggingface
!pip install langchain_community

In [149]:
!pip install langchain-openai


  Using cached tiktoken-0.11.0-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
Using cached tiktoken-0.11.0-cp312-cp312-win_amd64.whl (884 kB)

   ---------------------------------------- 0/2 [tiktoken]
   -------------------- ------------------- 1/2 [langchain-openai]
   -------------------- ------------------- 1/2 [langchain-openai]
   ---------------------------------------- 2/2 [langchain-openai]



In [61]:
!pip install openai

In [62]:
from dotenv import load_dotenv
load_dotenv()

True

In [63]:
import chromadb

chroma_client = chromadb.Client()

In [120]:
# Chroma에서 collection 삭제하기
chroma_client.delete_collection("tour")


In [121]:
collection = chroma_client.create_collection(name='tour')

In [122]:
import preprocess_functions as pf

documents = pf.make_csv_to_documents_with('전국 투어 데이터(숙박음식점제외).csv')

d:\SKN\3차 단위 프로젝트\rag관련\preprocess_functions.py:70: DtypeWarning: Columns (2,55,63,64,69,70,72,73,74,75,77,79,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv(file_path)


In [135]:
print(documents[0])

page_content='광주의 사직단은 광주 양림동 광주광역시립사직도서관 위쪽 사직공원에 있는 조선시대의 제단이다. 사직단은 신성한 장소로, 사()는 땅의 신, 직()은 곡식의 신을 가리킨다. 사직단은 땅의 신과 곡식의 신에게 제사를 올리며 나라의 안녕과 풍요를 기원하는 사직제를 지내는 공간이다. 사직단은 종묘와 함께 신성시되어 종사()라고도 했다. 사직단은 서울뿐 아니라 각 군현에도 있었으며, 기곡제, 기우제, 기청제, 기설제 등을 지냈다. 광주 사직단 크기는 서울 사직단 규모(2장 5척의 정사각형, 높이 3척)의 반 정도이다. 갑오년 이후 폐지되었으며, 한말에는 사직단을 군사 훈련장으로 사용하기도 했었다. 1971년 동물원이 들어서면서 그 자취가 없어졌는데, 1993년 광주시가 사직단을 복원, 1994년 광주 향우회 회장(최기영)에 의해 100년 만에 사직제가 부활되었다. 국토와 오곡은 민생의 근본이므로 삼국 시대부터 국태민안을 기원하는 사직단을 설치하여 친히 왕이 나가서 제사를 올렸다. 지방에서는 성주가 제주가 되었으므로 광주 목사가 매년 제사를 주관하였다.' metadata={'place': '사직단', '홈페이지': 'https://tour.gwangju.go.kr/', '주소': '광주광역시 남구 사직길 49 (사동)', '이용시간': '상시 개방', '휴일': '연중무휴', '주차': '불가능', '화장실': '장애인 화장실 있음(사직공원 공중화장실)', '입장료': '무료', '접근로': '사직단까지 경사로가 설치되어 있음_무장애 편의시설'}


In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


# 텍스트 정보를 포함한 documents 리스트
text_documents = [doc.page_content for doc in documents]

# 텍스트 분할기 생성 (청킹할 크기 지정)
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 문서별로 청크 나누기
chunks = []
for doc in text_documents:
    chunks.extend(text_splitter.split_text(doc))

# 임베딩 모델
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 청크된 텍스트에 대해 임베딩 생성
embeddings = embedding_model.encode(chunks)

# 각 청크에 고유 ID 부여
ids = [f"chunk_{i}" for i in range(len(chunks))]

# 메타데이터는 원본 문서 정보에서 가져옴 (청크별로 메타데이터를 추가할 수 있음)
metadatas = []
for doc in documents:
    # 각 문서에 대해 청크 수만큼 메타데이터를 추가
    metadatas.extend([doc.metadata] * len(text_splitter.split_text(doc.page_content)))

vectorstore = Chroma(collection_name="tour", embedding_function=embedding_model.encode)

# 배치 크기 설정
batch_size = 5000  # 최대 배치 크기

# 벡터 DB에 청크된 문서 추가 (배치 처리)
for i in range(0, len(chunks), batch_size):
    batch_chunks = chunks[i:i + batch_size]
    batch_embeddings = embeddings[i:i + batch_size]
    batch_metadatas = metadatas[i:i + batch_size]
    batch_ids = ids[i:i + batch_size]

    # 각 배치마다 벡터 DB에 추가
    collection.add(
        documents=batch_chunks,
        metadatas=batch_metadatas,
        embeddings=batch_embeddings,
        ids=batch_ids
    )

In [151]:
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="place",
        description="관광지 장소의 이름.",
        type="string",
    ),
    AttributeInfo(
        name="홈페이지",
        description="관광지 장소의 공식 웹사이트.",
        type="string",
    ),
    AttributeInfo(
        name="주소",
        description="관광지 장소의 주소.",
        type="string",
    ),
    AttributeInfo(
        name="이용시간",
        description="관광지 장소가 개방되는 시간.",
        type="string",
    ),
    AttributeInfo(
        name="휴일",
        description="관광지 장소가 휴무인 날.",
        type="string",
    ),
    AttributeInfo(
        name="주차",
        description="관광지 장소의 주차 가능 여부에 대한 정보.",
        type="string",
    ),
    AttributeInfo(
        name="화장실",
        description="관광지의 화장실 시설에 대한 정보, 장애인 화장실 포함.",
        type="string",
    ),
    AttributeInfo(
        name="입장료",
        description="관광지 입장료에 대한 정보.",
        type="string",
    )
]

# 지역별 관광지 정보
region_info = [
    {"name": "서울", "details": "서울은 대한민국의 수도로, 고대와 현대가 조화를 이루는 도시입니다. 유명한 명소로는 경복궁, N서울타워, 명동 등이 있습니다."},
    {"name": "부산", "details": "부산은 대한민국 제2의 도시로, 해운대, 광안리 해변, 자갈치 시장 등 해양 관광이 유명합니다."},
    {"name": "대구", "details": "대구는 전통과 현대가 어우러진 도시로, 대구 근대골목, 팔공산 등이 유명하며, 매년 대구국제뮤지컬페스티벌이 열립니다."},
    {"name": "인천", "details": "인천은 수도권에 위치한 항구도시로, 인천 차이나타운과 송도 국제도시가 유명한 명소입니다."},
    {"name": "광주", "details": "광주는 한국의 문화와 예술의 중심지로, 광주비엔날레와 5.18 민주화운동 기념관이 주요 관광지입니다."},
    {"name": "대전", "details": "대전은 과학의 도시로 알려져 있으며, 대덕연구단지와 국립중앙과학관 등이 있습니다."},
    {"name": "울산", "details": "울산은 대한민국의 산업도시로, 현대자동차와 울산항이 유명하며, 울산 대공원이 인기 관광지입니다."},
    {"name": "세종", "details": "세종시는 대한민국의 행정수도로, 정부청사가 있는 도시이며, 자연 경관과 함께 살기 좋은 도시로 알려져 있습니다."},
    {"name": "경기", "details": "경기도는 서울을 둘러싸고 있는 지역으로, 수원 화성, DMZ 등 다양한 역사적 명소와 자연을 제공합니다."},
    {"name": "강원", "details": "강원도는 아름다운 자연경관을 자랑하는 지역으로, 설악산, 평창 등의 겨울 스포츠와 자연을 즐길 수 있습니다."},
    {"name": "충북", "details": "충청북도는 산과 호수가 많아 자연경관이 뛰어난 지역으로, 청주와 제천이 주요 도시입니다."},
    {"name": "충남", "details": "충청남도는 바다와 산이 어우러지는 지역으로, 서해안과 태안해안국립공원이 유명합니다."},
    {"name": "경북", "details": "경상북도는 역사적인 유산이 풍부한 지역으로, 경주와 안동, 팔공산 등이 유명한 관광지입니다."},
    {"name": "경남", "details": "경상남도는 바다와 산이 아름다운 곳으로, 통영, 거제도, 산청 등이 유명한 관광지입니다."},
    {"name": "전북", "details": "전라북도는 전통적인 문화와 자연이 어우러지는 지역으로, 전주한옥마을, 고창, 무주가 유명합니다."},
    {"name": "전남", "details": "전라남도는 아름다운 해변과 섬들이 많은 지역으로, 여수, 순천, 담양 등이 주요 관광지입니다."},
    {"name": "제주", "details": "제주도는 대한민국의 대표적인 관광지로, 한라산, 성산일출봉, 만장굴 등이 유명한 자연 경관을 자랑합니다."}
]

document_content = ""
for region in region_info:
    document_content += f"장소명: {region['name']}\n상세 설명: {region['details']}\n\n"

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# 1. HuggingFace 모델을 사용한 임베딩 함수 설정
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"  # 사용한 모델 이름
embedding = HuggingFaceEmbeddings(model_name=embedding_model)

# 2. Chroma 벡터 DB 설정
vectorstore = Chroma(collection_name="tour", embedding_function=embedding)

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. retriever 객체 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents=document_content,
    metadata_field_info=metadata_field_info,
)

# 4. 사용자 질의 (예시)
query = "전남 가족여행 장소 추천"

# 5. retriever로 유사한 문서 검색
results = retriever.get_relevant_documents(query)



검색된 문서:


In [166]:
# OpenAI GPT 모델 사용 (예: GPT-3.5)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# 검색된 문서들을 하나의 문자열로 합쳐서 프롬프트에 전달
documents_text = ""
for result in results:
    # 결과에서 'place'와 'page_content'가 있는지 확인하고 추가
    place = result.metadata.get('place', 'Unknown Place')  # 'place'가 없으면 'Unknown Place'
    description = result.page_content if result.page_content else 'No description available.'  # 설명이 없으면 기본 텍스트 추가
    documents_text += f"장소: {place}\n설명: {description}\n\n"

# 프롬프트 템플릿
response_prompt = '''
당신은 여행 전문가입니다. 
다음 장소들을 바탕으로 {query}에 대해 친절하고 자세하게 추천을 해주세요.
{documents} 내용의 일부를 응답에 포함시켜 더 풍부한 답변을 생성해주세요.
답변을 가능한 한 세심하고 친절하게 작성해주세요. 각 여행지에 대한 간략한 설명도 포함해주세요. 
장소와 설명 구분: 검색된 문서에서 장소명과 설명을 구분하여 출력하고, 이를 바탕으로 여행지 추천을 제공해주세요.
모델은 각 장소에 대해 친절하게 여행지 추천 멘트를 생성하고, 이를 통해 사용자가 유용한 정보를 얻을 수 있도록 하세요.
3가지의 장소를 추천해주세요
'''

# 프롬프트 템플릿에 쿼리와 문서 텍스트 전달
prompt = response_prompt.format(query=query, documents=documents_text)

# LLM으로 응답 생성
response = llm.predict(prompt)

# 응답 출력
print("\nAI가 추천한 여행지:")
print(response)


AI가 추천한 여행지:
장소: 여수 해상케이블카
설명: 여수의 대표적인 관광 명소인 여수 해상케이블카는 여수 항과 오동도를 잇는 케이블카로, 아름다운 해안 풍경을 감상할 수 있는 곳입니다.

추천 멘트: 가족 모두가 함께 즐길 수 있는 여수 해상케이블카는 멋진 풍경을 감상하며 즐거운 시간을 보낼 수 있는 좋은 장소입니다. 특히 아이들이 좋아할 만한 환상적인 경치를 즐길 수 있어 추천드립니다.

---

장소: 무등산 국립공원
설명: 전남에 위치한 무등산 국립공원은 아름다운 자연 풍경을 감상할 수 있는 대표적인 산림욕지로, 다양한 산책로와 등산코스가 마련되어 있습니다.

추천 멘트: 가족 모두가 함께 자연 속에서 힐링할 수 있는 무등산 국립공원은 신선한 공기를 마시며 즐거운 시간을 보낼 수 있는 좋은 장소입니다. 아이들과 함께 가볍게 산책하거나 가족끼리 등산을 즐기기에 안성맞춤인 곳이에요.

---

장소: 순천만 갈대밭
설명: 순천에 위치한 순천만 갈대밭은 국내 최대 규모의 갈대밭으로, 아름다운 갈대밭 풍경을 감상할 수 있는 관광명소입니다.

추천 멘트: 순천만 갈대밭은 가족 모두가 함께 아름다운 자연 풍경을 감상하며 사진을 찍기에 좋은 장소입니다. 특히 봄 가을에 방문하면 더욱 멋진 풍경을 만날 수 있어 추천드립니다. 가족끼리 함께 힐링하고 즐거운 추억을 만들기에 딱 좋은 여행지입니다.


In [ ]:
# retriever 객체 생성
retriever = vectorstore.as_retriever()

# 사용자 질의 (두 쿼리를 동일하게 설정)
query = "아이와 가기 좋은 여행지 추천해줘"  

# retriever로 유사한 문서 검색
results = retriever.get_relevant_documents(query)

# 결과 출력 (여기서 검색된 문서들이 잘 나오는지 확인)
print("검색된 문서:")
for result in results:
    print(result.page_content)

# OpenAI GPT 모델 사용 (예: GPT-3.5)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# 검색된 문서들을 하나의 문자열로 합쳐서 프롬프트에 전달
documents_text = ""
for result in results:
    # 결과에서 'place'와 'page_content'가 있는지 확인하고 추가
    place = result.metadata.get('place', 'Unknown Place')  # 'place'가 없으면 'Unknown Place'
    description = result.page_content if result.page_content else 'No description available.'  # 설명이 없으면 기본 텍스트 추가
    documents_text += f"장소: {place}\n설명: {description}\n\n"

# 프롬프트 템플릿
response_prompt = '''
당신은 여행 전문가입니다. 
다음 장소들을 바탕으로 {query}에 대해 친절하고 자세하게 추천을 해주세요.
{documents}
답변을 가능한 한 세심하고 친절하게 작성해주세요. 각 여행지에 대한 간략한 설명도 포함해주세요. 장소와 설명 구분: 검색된 문서에서 장소명과 설명을 구분하여 출력하고, LLM이 이를 바탕으로 여행지 추천을 제공하는 방식입니다.

모델은 각 장소에 대해 친절하게 여행지 추천 멘트를 생성하고, 이를 통해 사용자에게 더 유용한 정보를 제공합니다.
'''

# 프롬프트 템플릿에 쿼리와 문서 텍스트 전달
prompt = response_prompt.format(query=query, documents=documents_text)

# LLM으로 응답 생성
response = llm.predict(prompt)

# 응답 출력
print("\nAI가 추천한 여행지:")
print(response)

검색된 문서:
예술적 체험과 감동을 맛볼 기회를 제공하는 공간
문암생태공원 내 반려동물과 보호자가 함께 쉴 수 있는 공간
장성문화원은 다양한 문화활동을 통해 지역 문화발전에 기여하고 있다.
올가리조트에서는 유기농 음료와 카이저 돔 생맥주를 즐길 수 있으며 넓은 잔디 정원과 쉴 수 있는 공간이 마련되어 있다.

AI가 추천한 여행지:
장소: 잠월미술관
설명: 잠월미술관은 예술적인 경험과 감동을 느낄 수 있는 공간으로, 다채로운 작품들을 감상하며 즐거운 시간을 보내실 수 있습니다. 부모님과 함께 방문하면 예술에 대한 새로운 시선을 찾을 수 있을 것입니다.

장소: 청주시 반려견 놀이터
설명: 반려동물과 함께 즐길 수 있는 문암생태공원 내 반려견 놀이터는 가족과 함께 휴식을 취할 수 있는 이색적인 장소입니다. 부모님과 반려동물을 데리고 가면 편안한 시간을 보내실 수 있을 것입니다.

장소: 장성문화원
설명: 장성문화원은 지역 문화 발전에 기여하는 다양한 문화 활동을 펼치고 있는 곳입니다. 부모님과 함께 방문하여 지역의 다양한 문화에 대해 배우고 체험하며 소중한 추억을 만들어보세요.

장소: 올가리조트
설명: 올가리조트는 유기농 음료와 생맥주를 즐길 수 있는 공간으로, 넓은 잔디 정원과 아늑한 분위기가 특징입니다. 부모님과 함께 힐링을 즐기고 싶다면 올가리조트를 방문해보세요. 풍부한 자연과 함께 휴식을 취할 수 있을 것입니다.
